
![Alt text](https://analyticsindiamag.com/wp-content/uploads/2020/08/2-lessthan-100kb.jpg)

## **Overview**

Welcome to another exciting weekend hackathon to flex your machine learning classification skills by classifying various classes of flowers into 8 different classes. To recognize the right flower you will be using 6 different attributes to classify them into the right set of classes(0-7). Using computer vision to do such recognition has reached state-of-the-art. Collecting Image data needs lots of human labor to annotate the images with the labels/bounding-boxes for detection/segmentation based tasks. Hence, some generic attribute which can be collected easily from various Area/ Locality/ Region were captured for over various species of flowers. 

In this hackathon, we are challenging the machinehack community to use classical machine learning classification techniques to come up with a machine learning model that can generalize well on the unseen data provided explanatory attributes about the flower species instead of a picture.

In this competition, you will be learning advanced classification techniques, handling higher cardinality categorical variables, and much more. 

## **Dataset Description:**

* Train.csv - 12666 rows x 7 columns (includes Class as target column)
* Test.csv - 29555 rows x 6 columns
* Sample Submission.csv - Please check the Evaluation section for more details on how to generate a valid submission.
 

## **Attributes Description:**

* Area_Code - Generic Area code, species were collected from
* Locality_Code - Locality code, species were collected from
* Region_Code - Region code, species were collected from
* Height - Height collected from lab data
* Diameter - Diameter collected from lab data
* Species - Species of the flower
* Class - Target Column (0-7) classes

 

In [6]:
pip install catboost

In [24]:
#Importing Required Libraries
#_______________________________________________________________________________________________________________
import numpy as np
import gc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size

# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Change column display number during print
pd.set_option('display.max_columns', 500)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
#Optimizing the Neural Network
from keras import optimizers
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.random.seed(9) # for reproducibility
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,StratifiedKFold
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.layers import BatchNormalization
from catboost import CatBoostClassifier

In [25]:
from google.colab import files
uploaded = files.upload()

Saving Train.csv to Train (1).csv


In [26]:
import io
train= pd.read_csv(io.BytesIO(uploaded['Train.csv']))
train

,Area_Code,Locality_Code,Region_Code,Height,Diameter,Class,Species
0,4694,17,1609,3.00,5.00,7,48
1,781,7,1380,4.00,17.00,2,54
2,21014,6,1122,3.00,11.00,1,22
3,7326,6,556,3.00,34.00,7,25
4,13122,17,1752,5.00,16.00,2,128
...,...,...,...,...,...,...,...
12661,28137,12,1133,3.00,21.00,2,181
12662,1524,12,1455,3.00,17.00,2,193
12663,2379,16,2175,11.00,30.00,6,22
12664,16200,17,1435,5.50,14.00,2,128


In [27]:
from google.colab import files
uploaded = files.upload()

Saving Test.csv to Test (1).csv


In [28]:
#Test Data
test= pd.read_csv(io.BytesIO(uploaded['Test.csv']))
test

,Area_Code,Locality_Code,Region_Code,Height,Diameter,Species
0,25836,4,903,7.50,8.00,14
1,28544,14,1576,3.00,7.00,48
2,7037,11,350,15.00,57.00,56
3,20460,17,1208,3.00,6.00,180
4,17555,14,994,4.50,22.00,225
...,...,...,...,...,...,...
29550,20833,1,1170,5.50,11.00,77
29551,28896,12,834,3.00,9.00,156
29552,2193,20,2169,7.50,32.00,49
29553,25195,10,1425,8.00,35.00,28


## **Adding New Features**

---



In [29]:
train['avg_diameter_species']=train.groupby('Species')['Diameter'].transform('mean')
train['avg_Height_species']=train.groupby('Species')['Height'].transform('mean')

train['feature_3']=train.groupby('Region_Code')['Diameter'].transform('mean')
train['feature_4']=train.groupby('Region_Code')['Height'].transform('mean')

train['feature_5']=train.groupby('Locality_Code')['Diameter'].transform('mean')
train['feature_6']=train.groupby('Locality_Code')['Height'].transform('mean')


test['avg_diameter_species']=test.groupby('Species')['Diameter'].transform('mean')
test['avg_Height_species']=test.groupby('Species')['Height'].transform('mean')

test['feature_3']=test.groupby('Region_Code')['Diameter'].transform('mean')
test['feature_4']=test.groupby('Region_Code')['Height'].transform('mean')

test['feature_5']=test.groupby('Locality_Code')['Diameter'].transform('mean')
test['feature_6']=test.groupby('Locality_Code')['Height'].transform('mean')


In [30]:
def my_groupby(df,primary_keys,dictionary_ops,renaming_dict):
    '''
        primary_keys is a list of primary keys.
        dictionary_ops is the dictionay having the operations to be performed (example :- {'location_number':'count'})
        renaming_dict is the column to be renamed after joining and resetting index
    '''
    return df.groupby(primary_keys).agg(dictionary_ops).reset_index().rename(columns=renaming_dict)

def data_left_join(df1,df2,primary_key):
    '''
        df1 :- First dataframe
        df2 :- Second Dataframe
        primary_key :- The list of primary keys on which one needs to left join
    '''
    return df1.merge(df2,how='left',on=primary_key)

def updated_df(df,primary_key,operation,columns):
    for cols in columns:
        print('Aggregate ',operation ,' on column- ',cols)
        df       = data_left_join(df,
                                   my_groupby(df,
                                              [primary_key],
                                              {cols:operation},
                                              {cols:primary_key+'_'+operation+'_'+cols}),
                                   primary_key)

    return df
train = updated_df(train,'Locality_Code','std',['Height','Diameter'])
test= updated_df(test,'Locality_Code','std',['Height','Diameter'])


train = updated_df(train,'Region_Code','std',['Height','Diameter'])
test= updated_df(test,'Region_Code','std',['Height','Diameter'])


Aggregate  std  on column-  Height
Aggregate  std  on column-  Diameter
Aggregate  std  on column-  Height
Aggregate  std  on column-  Diameter
Aggregate  std  on column-  Height
Aggregate  std  on column-  Diameter
Aggregate  std  on column-  Height
Aggregate  std  on column-  Diameter


In [31]:
train.shape,test.shape
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

((12666, 17), (29555, 16))

## **Modelling** 

In [33]:
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm


SEED = 42

FOLD=10
X=train.drop('Class',1)
y=train['Class']



Xt=test.copy()
test_preds = np.zeros((len(Xt),8))

# k-fold cross validation
scores=list()
kf = KFold(n_splits=FOLD, shuffle=True,random_state=SEED)

# enumerate splits

for train_idx, test_idx in tqdm(kf.split(X, y)):
    # get data
    X_train, y_train = X.iloc[train_idx, :], y[train_idx]
    X_test, y_test = X.iloc[test_idx, :], y[test_idx]
    test=Xt
    # fit model
    model=CatBoostClassifier(random_state=9,n_estimators=500)

    model.fit(X_train,y_train,cat_features=['Area_Code','Locality_Code','Species','Region_Code'])
    # evaluate model
    yhat =(model.predict_proba(X_test))
    test_preds +=(model.predict_proba(test))/FOLD
    acc=log_loss(y_test,yhat)
    
    
    # store score
    scores.append(acc)
    print('> ', acc)
    
# summarize model performance
# summarize model performance
mean_s= np.mean(scores)
std_s=np.std(scores)
print(model)
print('Mean: %.3f' % (mean_s))
print('std: %.3f' % (std_s))

Learning rate set to 0.160914
0:	learn: 1.6994538	total: 246ms	remaining: 2m 2s
1:	learn: 1.5232307	total: 499ms	remaining: 2m 4s
2:	learn: 1.3927580	total: 751ms	remaining: 2m 4s
3:	learn: 1.2977234	total: 981ms	remaining: 2m 1s
4:	learn: 1.2264315	total: 1.23s	remaining: 2m 2s
5:	learn: 1.1724833	total: 1.46s	remaining: 1m 59s
6:	learn: 1.1296763	total: 1.68s	remaining: 1m 58s
7:	learn: 1.0944586	total: 1.91s	remaining: 1m 57s
8:	learn: 1.0631969	total: 2.15s	remaining: 1m 57s
9:	learn: 1.0353155	total: 2.38s	remaining: 1m 56s
10:	learn: 1.0160793	total: 2.61s	remaining: 1m 55s
11:	learn: 0.9971601	total: 2.84s	remaining: 1m 55s
12:	learn: 0.9818132	total: 3.09s	remaining: 1m 55s
13:	learn: 0.9657046	total: 3.34s	remaining: 1m 55s
14:	learn: 0.9524563	total: 3.57s	remaining: 1m 55s
15:	learn: 0.9401976	total: 3.81s	remaining: 1m 55s
16:	learn: 0.9310060	total: 4.05s	remaining: 1m 54s
17:	learn: 0.9237772	total: 4.22s	remaining: 1m 52s
18:	learn: 0.9169117	total: 4.41s	remaining: 1m 5

>  0.747672490115511
Learning rate set to 0.160914
0:	learn: 1.7083636	total: 247ms	remaining: 2m 3s
1:	learn: 1.5276990	total: 492ms	remaining: 2m 2s
2:	learn: 1.3880543	total: 731ms	remaining: 2m 1s
3:	learn: 1.2898255	total: 971ms	remaining: 2m
4:	learn: 1.2173670	total: 1.22s	remaining: 2m 1s
5:	learn: 1.1598225	total: 1.46s	remaining: 2m
6:	learn: 1.1108844	total: 1.7s	remaining: 1m 59s
7:	learn: 1.0703829	total: 1.94s	remaining: 1m 59s
8:	learn: 1.0363353	total: 2.18s	remaining: 1m 59s
9:	learn: 1.0124799	total: 2.43s	remaining: 1m 58s
10:	learn: 0.9877944	total: 2.67s	remaining: 1m 58s
11:	learn: 0.9661974	total: 2.88s	remaining: 1m 56s
12:	learn: 0.9505325	total: 3.06s	remaining: 1m 54s
13:	learn: 0.9400771	total: 3.28s	remaining: 1m 53s
14:	learn: 0.9281541	total: 3.51s	remaining: 1m 53s
15:	learn: 0.9197501	total: 3.73s	remaining: 1m 52s
16:	learn: 0.9133572	total: 3.92s	remaining: 1m 51s
17:	learn: 0.9035901	total: 4.16s	remaining: 1m 51s
18:	learn: 0.8961403	total: 4.43s	re

>  0.7804895939948013
Learning rate set to 0.160914
0:	learn: 1.6994457	total: 239ms	remaining: 1m 59s
1:	learn: 1.5158154	total: 480ms	remaining: 1m 59s
2:	learn: 1.3923888	total: 733ms	remaining: 2m 1s
3:	learn: 1.2997335	total: 987ms	remaining: 2m 2s
4:	learn: 1.2286204	total: 1.23s	remaining: 2m 1s
5:	learn: 1.1773412	total: 1.46s	remaining: 2m
6:	learn: 1.1352429	total: 1.69s	remaining: 1m 58s
7:	learn: 1.0941664	total: 1.92s	remaining: 1m 58s
8:	learn: 1.0639693	total: 2.17s	remaining: 1m 58s
9:	learn: 1.0392175	total: 2.41s	remaining: 1m 58s
10:	learn: 1.0123487	total: 2.66s	remaining: 1m 58s
11:	learn: 0.9910040	total: 2.9s	remaining: 1m 58s
12:	learn: 0.9712240	total: 3.13s	remaining: 1m 57s
13:	learn: 0.9543217	total: 3.36s	remaining: 1m 56s
14:	learn: 0.9407180	total: 3.58s	remaining: 1m 55s
15:	learn: 0.9326772	total: 3.82s	remaining: 1m 55s
16:	learn: 0.9234943	total: 4.05s	remaining: 1m 55s
17:	learn: 0.9181102	total: 4.23s	remaining: 1m 53s
18:	learn: 0.9100879	total: 4.

>  0.7890267649791908
Learning rate set to 0.160914
0:	learn: 1.7086266	total: 245ms	remaining: 2m 2s
1:	learn: 1.5046248	total: 478ms	remaining: 1m 59s
2:	learn: 1.3786502	total: 726ms	remaining: 2m
3:	learn: 1.2888378	total: 1.01s	remaining: 2m 4s
4:	learn: 1.2207672	total: 1.24s	remaining: 2m 2s
5:	learn: 1.1670652	total: 1.46s	remaining: 2m
6:	learn: 1.1177445	total: 1.71s	remaining: 2m
7:	learn: 1.0821448	total: 1.94s	remaining: 1m 59s
8:	learn: 1.0495224	total: 2.19s	remaining: 1m 59s
9:	learn: 1.0265385	total: 2.45s	remaining: 1m 59s
10:	learn: 1.0031733	total: 2.67s	remaining: 1m 58s
11:	learn: 0.9839145	total: 2.9s	remaining: 1m 58s
12:	learn: 0.9694744	total: 3.13s	remaining: 1m 57s
13:	learn: 0.9606398	total: 3.25s	remaining: 1m 53s
14:	learn: 0.9474725	total: 3.5s	remaining: 1m 53s
15:	learn: 0.9363191	total: 3.7s	remaining: 1m 51s
16:	learn: 0.9279302	total: 3.92s	remaining: 1m 51s
17:	learn: 0.9206957	total: 4.11s	remaining: 1m 50s
18:	learn: 0.9116732	total: 4.35s	remain

>  0.7779785487594224
Learning rate set to 0.160914
0:	learn: 1.7128658	total: 243ms	remaining: 2m 1s
1:	learn: 1.5253032	total: 480ms	remaining: 1m 59s
2:	learn: 1.3923887	total: 738ms	remaining: 2m 2s
3:	learn: 1.3030199	total: 985ms	remaining: 2m 2s
4:	learn: 1.2280917	total: 1.23s	remaining: 2m 1s
5:	learn: 1.1721491	total: 1.46s	remaining: 2m
6:	learn: 1.1300845	total: 1.69s	remaining: 1m 59s
7:	learn: 1.0955267	total: 1.94s	remaining: 1m 59s
8:	learn: 1.0629545	total: 2.19s	remaining: 1m 59s
9:	learn: 1.0354258	total: 2.43s	remaining: 1m 58s
10:	learn: 1.0138666	total: 2.67s	remaining: 1m 58s
11:	learn: 0.9919585	total: 2.89s	remaining: 1m 57s
12:	learn: 0.9735356	total: 3.13s	remaining: 1m 57s
13:	learn: 0.9595896	total: 3.37s	remaining: 1m 57s
14:	learn: 0.9481628	total: 3.62s	remaining: 1m 56s
15:	learn: 0.9358145	total: 3.82s	remaining: 1m 55s
16:	learn: 0.9272748	total: 4.03s	remaining: 1m 54s
17:	learn: 0.9162842	total: 4.28s	remaining: 1m 54s
18:	learn: 0.9097187	total: 4.

...

>  0.7877356545734713
Learning rate set to 0.160915
0:	learn: 1.7403597	total: 235ms	remaining: 1m 57s
1:	learn: 1.5250535	total: 470ms	remaining: 1m 56s
2:	learn: 1.3921544	total: 704ms	remaining: 1m 56s
3:	learn: 1.2912099	total: 932ms	remaining: 1m 55s
4:	learn: 1.2218988	total: 1.16s	remaining: 1m 55s
5:	learn: 1.1684498	total: 1.39s	remaining: 1m 54s
6:	learn: 1.1230319	total: 1.62s	remaining: 1m 54s
7:	learn: 1.0814122	total: 1.85s	remaining: 1m 53s
8:	learn: 1.0504915	total: 2.09s	remaining: 1m 53s
9:	learn: 1.0284560	total: 2.32s	remaining: 1m 53s
10:	learn: 1.0046751	total: 2.54s	remaining: 1m 52s
11:	learn: 0.9852401	total: 2.77s	remaining: 1m 52s
12:	learn: 0.9669255	total: 3s	remaining: 1m 52s
13:	learn: 0.9529634	total: 3.21s	remaining: 1m 51s
14:	learn: 0.9415314	total: 3.44s	remaining: 1m 51s
15:	learn: 0.9329801	total: 3.62s	remaining: 1m 49s
16:	learn: 0.9224170	total: 3.87s	remaining: 1m 50s
17:	learn: 0.9118769	total: 4.1s	remaining: 1m 49s
18:	learn: 0.9053099	total

>  0.740687723057727
Learning rate set to 0.160915
0:	learn: 1.7367736	total: 249ms	remaining: 2m 4s
1:	learn: 1.5243278	total: 515ms	remaining: 2m 8s
2:	learn: 1.3996853	total: 755ms	remaining: 2m 5s
3:	learn: 1.3011128	total: 995ms	remaining: 2m 3s
4:	learn: 1.2259944	total: 1.24s	remaining: 2m 2s
5:	learn: 1.1656515	total: 1.49s	remaining: 2m 2s
6:	learn: 1.1246947	total: 1.72s	remaining: 2m 1s
7:	learn: 1.0882869	total: 1.94s	remaining: 1m 59s
8:	learn: 1.0555886	total: 2.19s	remaining: 1m 59s
9:	learn: 1.0306846	total: 2.42s	remaining: 1m 58s
10:	learn: 1.0083344	total: 2.63s	remaining: 1m 57s
11:	learn: 0.9866401	total: 2.87s	remaining: 1m 56s
12:	learn: 0.9705259	total: 3.1s	remaining: 1m 56s
13:	learn: 0.9563808	total: 3.31s	remaining: 1m 55s
14:	learn: 0.9469817	total: 3.44s	remaining: 1m 51s
15:	learn: 0.9371842	total: 3.64s	remaining: 1m 50s
16:	learn: 0.9281993	total: 3.86s	remaining: 1m 49s
17:	learn: 0.9184537	total: 4.09s	remaining: 1m 49s
18:	learn: 0.9091871	total: 4.3

>  0.6996879764521274
Learning rate set to 0.160915
0:	learn: 1.7365344	total: 230ms	remaining: 1m 54s
1:	learn: 1.5206193	total: 475ms	remaining: 1m 58s
2:	learn: 1.3871717	total: 717ms	remaining: 1m 58s
3:	learn: 1.2863298	total: 966ms	remaining: 1m 59s
4:	learn: 1.2186675	total: 1.21s	remaining: 1m 59s
5:	learn: 1.1579509	total: 1.44s	remaining: 1m 58s
6:	learn: 1.1210656	total: 1.7s	remaining: 1m 59s
7:	learn: 1.0867962	total: 1.94s	remaining: 1m 59s
8:	learn: 1.0529742	total: 2.18s	remaining: 1m 58s
9:	learn: 1.0283989	total: 2.41s	remaining: 1m 57s
10:	learn: 1.0039941	total: 2.64s	remaining: 1m 57s
11:	learn: 0.9816887	total: 2.87s	remaining: 1m 56s
12:	learn: 0.9671806	total: 3.09s	remaining: 1m 55s
13:	learn: 0.9563151	total: 3.33s	remaining: 1m 55s
14:	learn: 0.9469283	total: 3.53s	remaining: 1m 54s
15:	learn: 0.9368566	total: 3.73s	remaining: 1m 52s
16:	learn: 0.9251190	total: 3.95s	remaining: 1m 52s
17:	learn: 0.9181529	total: 4.16s	remaining: 1m 51s
18:	learn: 0.9097386	to

>  0.7518204634822119
Learning rate set to 0.160915
0:	learn: 1.7446218	total: 223ms	remaining: 1m 51s
1:	learn: 1.5316441	total: 455ms	remaining: 1m 53s
2:	learn: 1.4000113	total: 686ms	remaining: 1m 53s
3:	learn: 1.3079551	total: 919ms	remaining: 1m 53s
4:	learn: 1.2300033	total: 1.15s	remaining: 1m 53s
5:	learn: 1.1758468	total: 1.39s	remaining: 1m 54s
6:	learn: 1.1286377	total: 1.64s	remaining: 1m 55s
7:	learn: 1.0903517	total: 1.84s	remaining: 1m 53s
8:	learn: 1.0559602	total: 2.09s	remaining: 1m 54s
9:	learn: 1.0276059	total: 2.33s	remaining: 1m 54s
10:	learn: 1.0050192	total: 2.57s	remaining: 1m 54s
11:	learn: 0.9864477	total: 2.81s	remaining: 1m 54s
12:	learn: 0.9741573	total: 3s	remaining: 1m 52s
13:	learn: 0.9631952	total: 3.23s	remaining: 1m 52s
14:	learn: 0.9512859	total: 3.44s	remaining: 1m 51s
15:	learn: 0.9433229	total: 3.68s	remaining: 1m 51s
16:	learn: 0.9346199	total: 3.91s	remaining: 1m 51s
17:	learn: 0.9250072	total: 4.09s	remaining: 1m 49s
18:	learn: 0.9196483	tota

>  0.7315714499226521

Mean: 0.754
std: 0.028


## **Predictions on Test Data**

---



In [ ]:
test_preds=pd.DataFrame(test_preds)
test_preds.columns=['Class_0','Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7']
test_preds.to_csv('cat-3.csv',index=False)

In [ ]:
from google.colab import files
files.download('cat-3.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>